### Using the Poisson process to predict litter survey results on Lac Léman
#### Another bit of advice from Prof Davison was to use the Poisson process to answer the question:

> What are the chances of finding X quantity of some object on the beach (during a survey)

#### This is pretty important stuff in the sense that if this works then that means that the quantity of any specific item found per meter is random, which would make it difficult to predict or to estimate the efficiency of litter mitigation efforts over time.

#### Huh, what did he just say? Our basic assumption is that you have less chance finding an object in nature if their is less of it in nature ---- if that doesn't make sense I am trading my keyboard in for a console and the coffee for a beer

#### Load the libraries needed:

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import re
import os
from scipy.stats import norm
import scipy.stats
import statsmodels.api as sm #<----- this library has a bunch of tools to use on the poisson process
here = os.getcwd()
data = '/home/mw-shovel/web/notes/basel/data'
here

'/home/mw-shovel/web/notes/notes-hammerdirt.ch'

#### Get the data and any other 'objects' required

In [2]:
# in the note book we pickle everything for storage
# in the app we use json format, pickle is not secure for web apps (as far as I know)
slr = pd.read_pickle(data + '/combined.p')
mcb = pd.read_pickle(data + '/combined_mc.p')
mcb_slr = pd.read_pickle(data + '/mcb_slr.p')
g_all = pd.read_pickle(data + '/water_bodies_all.p')
g = pd.read_pickle(data + '/water_bodies.p')
codes = pd.read_pickle(data + '/codes.p')

In [3]:
mcb_slr.reset_index(inplace=True)

#### Run through the same checks and transformations as in the previous note book

In [4]:
# the records are stored as individual events per object type
# for example one record from the data base looks like this
mcb_slr.loc[1]

index                            1
code_id                        G27
date           2017-04-02 00:00:00
pcs_m                        0.429
length                          28
location_id     Aare_Bern_CaveltiN
quantity                        12
city                 Muri bei Bern
latitude                   46.9236
longitude                  7.47332
post                          3074
water                        river
pop                          13187
area                          7.63
pop_dens                   1728.31
day                              6
Name: 1, dtype: object

In [5]:
#need to combine records by date and location

dfA = mcb_slr.pivot_table('pcs_m', index=['date', 'day', 'water','city','location_id', 'length'], columns=['code_id'], fill_value=0)

In [6]:
# now all the records that have the same 'date', 'day', 'city' and 'location_id' are grouped together
# in other words all the results from one beach-litter inventory
dfA[:1]

,,,,,code_id,G1,G10,G100,G101,G102,G11,G12,G124,G125,G126,...,G89,G9,G90,G91,G92,G93,G95,G96,G97,G99
date,day,water,city,location_id,length,,,,,,,,,,,,,,,,,,,,,
2015-11-23,0,lake,Montreux,Baye de Montreux - G,61.0,0.0,0.0,0.016393,0.0,0.0,0.016393,0.0,0.016393,0.0,0.016393,...,0.065574,0.0,0.0,0.0,0.0,0.0,0.147541,0.0,0.0,0.0


In [7]:
# add up all the columns (those with a 'G' label)
dfA['total'] = dfA[dfA.columns].sum(axis=1)

In [8]:
# get the first 10 records for Lac Leman
idx = pd.IndexSlice
dfA.loc[idx[:, :, :, :, g_all['Lac Léman']], 'total'][:10]
# this looks good

date        day  water  city      location_id           length
2015-11-23  0    lake   Montreux  Baye de Montreux - G  61.0      5.721311
2015-11-24  1    lake   Montreux  Baye de Clarens       69.0      1.695652
2015-11-27  4    lake   Vevey     Veveyse               53.0      4.075472
2015-12-01  1    lake   Vevey     Veveyse               53.0      0.981132
2015-12-02  2    lake   Montreux  Baye de Clarens       69.0      0.681159
2015-12-04  4    lake   Montreux  Baye de Montreux - D  61.0      6.918033
                                  Baye de Montreux - G  61.0      8.377049
2015-12-07  0    lake   Vevey     Veveyse               53.0      3.641509
2015-12-08  1    lake   Montreux  Baye de Clarens       69.0      3.391304
2015-12-10  3    lake   Montreux  Baye de Montreux - D  61.0      9.655738
Name: total, dtype: float64

In [9]:
dfA.reset_index(inplace=True)

#### Get down to just what is needed

In [10]:
a = dfA.loc[dfA.location_id.isin(g_all['Lac Léman'])].copy()
# just to make sure its all good:
a[:5]
# its all good

code_id,date,day,water,city,location_id,length,G1,G10,G100,G101,...,G9,G90,G91,G92,G93,G95,G96,G97,G99,total
0,2015-11-23,0,lake,Montreux,Baye de Montreux - G,61.0,0.0,0.0,0.016393,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.147541,0.0,0.0,0.0,5.721311
1,2015-11-24,1,lake,Montreux,Baye de Clarens,69.0,0.0,0.0,0.000000,0.014493,...,0.0,0.0,0.0,0.0,0.0,0.014493,0.0,0.0,0.0,1.695652
2,2015-11-27,4,lake,Vevey,Veveyse,53.0,0.0,0.0,0.018868,0.018868,...,0.0,0.0,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,4.075472
3,2015-12-01,1,lake,Vevey,Veveyse,53.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.981132
4,2015-12-02,2,lake,Montreux,Baye de Clarens,69.0,0.0,0.0,0.014493,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.681159


### Should explain here how the Poisson distribution works --- is it process or distribution? Sort that out first.  

1. Poisson process: A non deterministic process where events occur continuosly and independent of each other.  This sounds like trash data.
2. Poisson distribution: A discrete probability distribution that represents the probability of events (that have a Poisson process) occuring in a certain space (time, distance)
3. Poisson distributions arises from situtations in whuch there is a large number of opportunitie for the event to occur, but a small chance that it will occur on any one trial.---- This maybe a non-starter

#### How did we get here? I asked if we could look at each record as an EVENT in a series of Bernouli trials --- thus using a Bernouli distribution to determine the probability of finding an item.  Turns out the Poisson distribution the sum of independent Bernouli trials.  ---- Doh! I was so close

In [11]:
# I know that says what it is not how it works
# there are some conditions that need to be met, they can all be found in the data or computed:
# 1) Events can be counted in whole numbers --- in the data above an event is one meter (pieces of garbage/meter)
# 2) Occurences are independent --- each piece of trash arrived there on its own
#    that is to say that me throwing my garbage in the lake does not influence your decision to throw
#    garbage in the lake -- (the broken window effect not included)
# 3) The average frequency of occurence for the period is known (avg pieces/meter)
# 4) It is possible to count how many events have occured

In [12]:
# try this out for one item category
# r is the average occurence
# k is the number of occurences for which we want to know the probaility
import math
def make_fish(k, r):
    # p(k) = r*k/(k!)(e*r)
    a = r**k
    b = math.factorial(k)
    e = math.exp(1)**r
    
    c = b*e
    d = a/c
    return d


In [13]:
code_avg = a['G95'].mean()#<---- actual average pieces per meter
trials = len(a)#<---- this the number of trials
p_g95 = make_fish(1, code_avg)#<------ Run the function and see what the probability of getting one piece per meter is
p_g95#<----- this output is very close to what you would expect ot find on the poisson table for p(1) and r or l = 0.5

0.3211248652886724

In [14]:
p_of_one = trials * p_g95 #<---- multiply the proablity by the number of events
p_of_one

44.95748114041414

In [15]:
a.loc[(a.G95 > 0.49) & (a.G95 < 1.5), 'G95'].count() #<------- actual number of events around 1

26

### This is not looking random, the actual number of times that the result was around 'one' is 26.

#### Either this being set up wrong or the results are not random.  Try one more thing:

1. Instead of doing all the locations on the lake do just one location


In [16]:
b = dfA.loc[dfA.location_id == 'Veveyse'].copy() #<---- selecting just veveyse

In [26]:
b1 = b[['G95', 'length']].copy()
b1['q'] = b1['G95']*b1['length']

In [35]:
b1

code_id,G95,length,q
2,0.018868,53.0,1.0
3,0.000000,53.0,0.0
7,0.037736,53.0,2.0
11,0.018868,53.0,1.0
18,0.037736,53.0,2.0
21,0.094340,53.0,5.0
24,0.207547,53.0,11.0
27,0.018868,53.0,1.0
31,0.075472,53.0,4.0
40,0.037736,53.0,2.0


In [46]:
len(b1.loc[b1.length == 53])

13

In [80]:
def make_fishes(df, k, code):
    # p(k) = r*k/(k!)(e*r)
    r =  df.loc[df.length == 53, code].mean()
    a = r**k
    b = math.factorial(k)
    e = math.exp(1)**(-r)
    c = b*e
    d = (a*e)/b
    f = df.loc[(df['length'] == 53) & (df[code] == 1)][code].count()
    g = d*17
    #print(df.loc[(df[code] > .1) & (df[code] < 1.5), code])
    return d, r, g, f

In [81]:
make_fishes(b1, 1, 'q')

(0.1028206059119018, 3.5384615384615383, 1.7479503005023307, 4)

In [73]:
b1.loc[(b1.length == 53) & (b1.q == 1)]['q'].count()

4

In [82]:
4/13

0.3076923076923077

In [28]:
def get_fishes(length, code):
    for c in code:
        b_l = b1.loc[b.length == length].copy()
        d, e, r, f = make_fishes(b_l, 1, c, length)
        print('the code is ' + str(c) + ' this ' + str(e) + ' should be close to the actual result: ' + str(f) + '\n')
        

In [29]:
get_fishes(53, ['q'])
# the output is like this:
# 1 -- matching output from the data (if any) --- Series([]) means there was no occurences in the data
# 2 -- the series indentifier "Name: G95, dtype: float64"
# 3 -- the code is G95 this {calculated result} should be close to the actual result: {actual result}

2     1.0
11    1.0
27    1.0
45    1.0
Name: q, dtype: float64
the code is q this 6453.936224657875 should be close to the actual result: 4



### The resluts are mixed, indicating that some of these objects are truly random occurences and others not so much? -- maybe

#### This will take a little more investigation

In [20]:
def find_it():
    for i in codes:
         for key, value in i.items():
                

SyntaxError: unexpected EOF while parsing (<ipython-input-20-49d10052f4e5>, line 4)